# Balanced Accuracy Demo
Bu defter GitHub üzerinde paylaşılmak üzere hazırlanmıştır. Dengesiz veri setlerinde **balanced accuracy** metriğinin neden daha doğru bir değerlendirme sunduğunu gösterir.

## Nasıl Çalıştırılır?
```bash
git clone <repo-url>
cd <repo>
pip install -r requirements.txt  # veya: pip install scikit-learn matplotlib
jupyter notebook balanced_accuracy_demo_enhanced.ipynb
```
Python ≥3.8 önerilir.


# Dengelenmiş Doğruluk (Balanced Accuracy)
Bu not defterinde **balanced accuracy** kavramını ayrıntılı olarak inceleyecek ve Python ile küçük bir örnek uygulayacağız.

## Tanım 
Balanced accuracy (dengelenmiş doğruluk), sınıf dağılımı dengesiz (imbalanced) olan sınıflandırma problemlerinde model performansını daha adil bir şekilde ölçmek için kullanılan bir metriktir. Her sınıf için **recall** (duyarlılık) değerini hesaplar ve bu değerlerin aritmetik ortalamasını alır.

$$ \text{Balanced Accuracy} = \frac{1}{C}\sum_{i=1}^{C}\frac{TP_i}{TP_i + FN_i} $$

Burada $C$ sınıf sayısıdır, $TP_i$ i'inci sınıf için gerçek pozitiflerin sayısı, $FN_i$ ise gerçek kaçırılan (false negative) örneklerin sayısıdır.

## Neden ve Ne Zaman Kullanılır?
- **Sınıf dengesizliği** olduğunda (`weights=[0.95, 0.05]` gibi), klasik accuracy metriği çoğunluk sınıfını tahmin ederek yüksek değer verebilir.
- Balanced accuracy, her sınıfın başarı oranını eşit derecede önemser; böylece azınlık sınıfın tahmin doğruluğunu gizlemez.
- Özellikle tıp, sahtekârlık tespiti ve güvenlik gibi azınlık sınıfın kritik olduğu senaryolarda tercih edilir.

## Örnek Çalışma 
Aşağıdaki hücrelerde yapay olarak dengesiz bir veri seti oluşturacağız (%90 negatif, %10 pozitif). Bir Lojistik Regresyon modeli eğitip hem klasik accuracy hem de balanced accuracy hesaplayacağız.

In [ ]:
# Gerekli kütüphaneleri içe aktar
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# 1) Yapay veri seti oluşturalım
# Bu veri seti iki sınıflı olacak ve sınıflar arasında dengesizlik olacak (örn: %90 sınıf 0, %10 sınıf 1)
X, y = make_classification(n_samples=5000,       # toplam 5000 örnek
                           n_features=20,         # 20 özellik (bağımsız değişken)
                           n_informative=3,       # 3 tanesi gerçekten bilgi içeriyor
                           n_redundant=0,         # gereksiz (tekrarlı) özellik yok
                           n_clusters_per_class=1,
                           weights=[0.9, 0.1],    # %90 sınıf 0, %10 sınıf 1
                           random_state=42)       # tekrarlanabilirlik için sabit tohum

# 2) Eğitim ve test verilerini ayıralım
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)  # %70 eğitim, %30 test

# 3) Lojistik regresyon modeli oluşturalım ve eğitelim
clf = LogisticRegression(max_iter=1000)  # maksimum iterasyon sayısını artırıyoruz
clf.fit(X_train, y_train)  # modeli eğitiyoruz

# 4) Test verisi ile tahmin yapalım
y_pred = clf.predict(X_test)  # tahmin edilen etiketler

# 5) Performans metriklerini hesaplayalım
acc = accuracy_score(y_test, y_pred)  # klasik doğruluk (accuracy)
bal_acc = balanced_accuracy_score(y_test, y_pred)  # dengelenmiş doğruluk (balanced accuracy)

# Sonuçları yazdıralım
print(f"Klasik Accuracy: {acc:.4f}")
print(f"Balanced Accuracy: {bal_acc:.4f}")

# 6) Confusion matrix'i görselleştirelim
cm = confusion_matrix(y_test, y_pred)  # karışıklık matrisi
fig, ax = plt.subplots()
ConfusionMatrixDisplay(confusion_matrix=cm).plot(ax=ax)  # matrisi çizdir
plt.title('Confusion Matrix')
plt.show()

## 🚀 Sonraki Aşamalar (Ne Yapılabilir?)
- **class_weight='balanced'** veya `compute_class_weight` ile modeli tekrar eğitip azınlık sınıfa ağırlık verilebilir.
- **SMOTE** ya da **RandomOverSampler** gibi tekniklerle azınlık sınıfı örnekleri artırılabilir.
- Farklı modeller (Random Forest, Gradient Boosting, XGBoost, LightGBM) denenerek performans karşılaştırması yapılabilir.
- **ROC-AUC**, **PR-AUC**, **F1-score** gibi ilave metrikler eklenerek dengesiz veri setlerinde daha kapsamlı değerlendirme yapılabilir.
- Veri setini gerçek bir probleme (örn: sahtekârlık tespiti) entegre ederek iş bağlamı oluşturulabilir.

Aşağıdaki hücre `class_weight='balanced'` parametresinin etkisini hızlıca gösterir.

In [ ]:
# class_weight='balanced' ile yeniden eğitim
# Bu parametre, modelin azınlık sınıfa daha fazla önem vermesini sağlar.
# Böylece sınıf dengesizliğinin etkisi azaltılır.

clf_bal = LogisticRegression(max_iter=1000, class_weight='balanced')  # sınıflara ağırlık vererek model kur
clf_bal.fit(X_train, y_train)  # modeli eğit

y_pred_bal = clf_bal.predict(X_test)  # test verileri üzerinde tahmin yap

acc_bal = accuracy_score(y_test, y_pred_bal)  # klasik doğruluk (accuracy)
bal_acc_bal = balanced_accuracy_score(y_test, y_pred_bal)  # dengelenmiş doğruluk

print("\n--- class_weight='balanced' Sonuçları ---")
print(f"Accuracy: {acc_bal:.4f}")  # doğruluk oranını yazdır
print(f"Balanced Accuracy: {bal_acc_bal:.4f}")  # balanced accuracy oranını yazdır

# Confusion matrix ile sınıf bazında başarıyı görselleştir
cm_bal = confusion_matrix(y_test, y_pred_bal)
fig, ax = plt.subplots()
ConfusionMatrixDisplay(confusion_matrix=cm_bal).plot(ax=ax)
plt.title("Confusion Matrix (class_weight='balanced')")
plt.show()

---
Bu not defteri [MIT Lisansı](LICENSE) ile sunulmuştur. Katkılarınızı pull request veya issue yoluyla iletebilirsiniz.

_Son güncelleme: 2025-07-15 07:45_